![image.png](image/Problema_de_Transporte.png)

## Modelagem do Problema

Índices:
- $i=1, ..., m$ pontos de origem (oferta);
- $j=1, ..., n$ pontos de destino (demanda);

Parâmetros:
- $s_i$ quantidade ofertada na origem $i$;
- $d_j$ quantidade demandada no destino $j$;
- $c_{ij}$ custo unitário de transporte entre a origem $i$ e o destino $j$.

Objetivo:

$$
\min \sum_{i=1}^{m}\sum_{j=1}^{n}c_{ij}x_{ij}
$$

Restrições:

- Caso 1: O total ofertado é maior que o total demandado. Nesse caso, nem toda a oferta será enviada.

$$
\sum_{j=1}^{n}x_{ij} \leq s_i, \;\; \forall i=1, ..., m \\
\sum_{i=1}^{m}x_{ij} = d_j, \;\; \forall j=1, ..., n \\
x_{ij} \geq 0
$$

- Caso 2: O total demandado é maior que o total ofertado. Nesse caso, nem toda a demanda será atendida

$$
\sum_{j=1}^{n}x_{ij} = s_i, \;\; \forall i=1, ..., m \\
\sum_{i=1}^{m}x_{ij} \leq d_j, \;\; \forall j=1, ..., n \\
x_{ij} \geq 0
$$

In [9]:
import gurobipy as gp

In [10]:
# Parâmetros do problema

instancia = 2
if instancia == 1:
    # INSTÂNCIA 1 (Oferta maior que a demanda)
    vet_ofertas = [140, 160, 200, 190]
    vet_demandas = [50, 80, 30, 50, 100, 90, 20, 70, 120]
elif instancia == 2:
    # INSTÂNCIA 2 (Demanda maior que a oferta)
    vet_ofertas = [140, 160, 120, 190]
    vet_demandas = [50, 80, 30, 50, 100, 90, 120, 70, 120]
else:
    # INSTÂNCIA 3 (Demanda igual à oferta)
    vet_ofertas = [190, 160, 170, 190]
    vet_demandas = [50, 80, 30, 50, 100, 90, 120, 70, 120]

qtd_fabricas = len(vet_ofertas)
qtd_clientes = len(vet_demandas)    
    
oferta_total = sum(vet_ofertas)
demanda_total = sum(vet_demandas)
    
vet_custos = [[12, 25, 39, 17, 38, 40, 8, 25, 13],
              [17, 26, 20, 25, 30, 25, 14, 20, 15],
              [35, 15, 18, 20, 12, 42, 27, 26, 19],
              [28, 30, 37, 30, 28, 36, 16, 24, 32]]

print(f'Oferta total:{oferta_total} e demanda total: {demanda_total}')

Oferta total:610 e demanda total: 710


In [11]:
# Rótulos das fábricas e clientes
fabricas = [f'Fab_{i}' for i in range(1, qtd_fabricas+1) ]
clientes = [f'Cli_{i}' for i in range(1, qtd_clientes+1) ]

In [12]:
# Dicionários com as ofertas
ofertas = {fabricas[i]:valor for i, valor in enumerate(vet_ofertas)}

# Dicionários com as demandas
demandas = {clientes[i]:valor for i, valor in enumerate(vet_demandas)}

In [13]:
custos = {(fabricas[i], clientes[j]): vet_custos[i][j] for i in range(qtd_fabricas) for j in range(qtd_clientes)}

In [14]:
m = gp.Model()

# Variáveis de decisão
x = m.addVars(fabricas, clientes, vtype=gp.GRB.INTEGER)

# Função objetivo
m.setObjective(
    gp.quicksum(x[i, j] * custos[i, j] for i in fabricas for j in clientes),
    sense=gp.GRB.MINIMIZE)

# Restrições de oferta
if oferta_total > demanda_total:
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in clientes) <= ofertas[i] for i in fabricas)
else:
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in clientes) == ofertas[i] for i in fabricas)

# Restrições de demanda
if demanda_total > oferta_total:
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in fabricas) <= demandas[j] for j in clientes)
else:
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in fabricas) == demandas[j] for j in clientes)

# Executa o modelo
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 13 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x9a324ff4
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective 18940.000000
Presolve time: 0.00s
Presolved: 13 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (0 binary)

Root relaxation: objective 1.008000e+04, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    10080.000000 10080.0000  0.00%     -    0s

Explored 0 nodes (13 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 10080 18940 

Optimal solution found (tolerance 1.00e-04

In [17]:
# Imprime o plano de transporte das fábricas
for i in fabricas:
    print("Origem:", i)
    for j in clientes:
        qtd = round(x[i, j].X)
        if qtd > 0:
            print(f"Transportar {qtd} unidades para {j}")
    print("")

Origem: Fab_1
Transportar 50 unidades para Cli_1
Transportar 50 unidades para Cli_4
Transportar 40 unidades para Cli_7

Origem: Fab_2
Transportar 30 unidades para Cli_3
Transportar 10 unidades para Cli_6
Transportar 120 unidades para Cli_9

Origem: Fab_3
Transportar 20 unidades para Cli_2
Transportar 100 unidades para Cli_5

Origem: Fab_4
Transportar 40 unidades para Cli_2
Transportar 80 unidades para Cli_7
Transportar 70 unidades para Cli_8



In [19]:
# Relatório de oferta ou demanda desbalanceada
if oferta_total > demanda_total:
    print("As fábricas a seguir tem capacidade excedente:")
    for i in fabricas:
        sobra = round(c1[i].Slack)
        if sobra > 0:
            print(f"A Fábrica: {i} tem {sobra} unidades")
elif demanda_total > oferta_total:
    print("Os clientes a seguir não tiveram toda a demanda atendida:")
    for j in clientes:
        sobra = round(c2[j].Slack)
        if sobra > 0:
            print(f"Faltou {sobra} unidades a ser entregue ao Cliente: {j}")
else:
    print("A oferta e demanda estão balanceadas")

Os clientes a seguir não tiveram toda a demanda atendida:
Faltou 20 unidades a ser entregue ao Cliente: Cli_2
Faltou 80 unidades a ser entregue ao Cliente: Cli_6
